In [153]:
import pandas as pd
import numpy as np
import csv
import ast
from datetime import datetime
import collections

In [370]:
naver_review = pd.read_excel('/Users/dhkim/PycharmProjects/RealTastySpot/data/네이버리뷰/naver_review.xlsx')
kakao_review = pd.read_excel('/Users/dhkim/PycharmProjects/RealTastySpot/data/카카오리뷰/kakao_review.xlsx')
google_review = pd.read_excel('/Users/dhkim/PycharmProjects/RealTastySpot/data/구글리뷰/google_reviews.xlsx', sheet_name=1)
mango_review = pd.read_excel('/Users/dhkim/PycharmProjects/RealTastySpot/data/망고플레이트/mangoplates_reviews.xlsx')
dinning_review = pd.read_excel('/Users/dhkim/PycharmProjects/RealTastySpot/data/다이닝코드/dinning_review_reshape.xlsx')
dining_like = pd.read_excel('/Users/dhkim/PycharmProjects/RealTastySpot/data/다이닝코드/dinningcode_review_like.xlsx')

# mango_review, google_review 전처리

In [371]:
def preprocessing_allreview(df, seperate = "-"):
    df.dates = ''
    df.scores = ''
    df.reviews = ''
    for idx, review in enumerate(df.allreviews):
        date_dict = {}
        score_dict = {}
        review_dict = {}
        key_ = 0
        try:
            for i, values in enumerate([x.replace('{','').split(seperate) for x in review.split('}')]):
                if len(values)==1:
                    continue
                try:
                    if len(values) >= 3:
                        a, b, c = values[:3]
                        review_dict[key_] = c
                    else:
                        a, b = values
                        review_dict[key_] = ''
                    if '전' in b:
                        b = b.replace(' ', '')
                    b = b.replace('-', '.')
                    b = b.rstrip()
                    date_dict[key_] = b
                except Exception as e:
                    print(f'values : {values}')
                    print(f'idx={idx}, i={i} error :{e}, a= "{a.split(" ")[-2]}", b= "{b}", c= "{c}"')
                try:
                    score_dict[key_] = int(a.split(' ')[-2][0])
                except IndexError as e:
                    pass
#                     print(f'values : {values}')
#                     print(f'idx={idx}, i={i} error :{e}')
                key_ += 1

        except AttributeError as e:
            print(f'{idx}, {e}, {review}')

        df.loc[idx, 'dates'] = str(date_dict)
        df.loc[idx, 'scores'] = str(score_dict)
        df.loc[idx, 'reviews'] = str(review_dict)
    return df

## 구글 리뷰 seperator : "-"

In [372]:
google_review.loc[7, 'allreviews'].split('}')[-30].split('-')

['{527: 별표 1개 ',
 '1년 전',
 '고객님의 소중한 의견 감사합니다! 메뉴 보는게 조금 어려우셨군요 ㅠ..ㅠ..이 부분 수렴하여 개선할 수 있도록 노력하겠습니다. 감사합니다 :)']

## 망고 리뷰 seperator : ":"

In [373]:
mango_review.loc[7, 'allreviews'].split('}')[0].split(':')

['{1',
 '2020-08-20 ',
 '           20200609◇ 콩 국 수*. 인생 첫 콩국수...너무너무 대성공. 이럴수가. 저는 마카롱이나 콩국수나 처음 먹어보는 것들을 거의 <진수>라고 불리는 곳에서만 접해서 어쩐대요... 눈높이 너무 높아져서 이제 아무 콩국수나 못먹어벌여ㅠ*. 진짜 고소하고 꾸덕한 콩국수의 끝판왕이 아닐까요.. 콩을 워낙 좋아해도, 신문지맛이 난다는 소문 탓에 쉽사리 도전하지 못했는데... 역시 인간은 도전해야 경험하고 경험해야 진리를 찾을 수 있다는 말을 또 되새기게 됩니다...*. 그리고 여기 무슨 보쌈집인가요??? 김치 왜케 맛있어~!!~!! 김치 잘 안 먹는 저같은 어린이도 싹 비우게 하는 맛입니다.. 이 김치 하나만 있으면 쌀밥 한 그릇도 뚝딱 비워내겠는걸요. 물론 전 쌀밥은 안 먹긴 하지만요..        ']

In [374]:
preprocessing_allreview(google_review, seperate='-')
google_review.loc[:,'review_len'] = google_review['dates'].map(lambda x:len(x.split(',')))

In [375]:
preprocessing_allreview(mango_review, seperate=':')
mango_review.loc[:, 'review_len'] = mango_review['dates'].map(lambda x:len(x.split(',')))

# 각 사이트 별 리뷰 개수 구하기 
- 최근 1년 리뷰 개수, 최근 6개월 리뷰 개수, 최근 3개월 리뷰 개수, 최근 1개월 리뷰 개수

In [376]:

def preprocess_datetime(df:pd.DataFrame, site:str='naver'):
    df[f'{site}_1Y_num'] = ''
    df[f'{site}_6M_num'] = ''
    df[f'{site}_3M_num'] = ''
    df[f'{site}_1M_num'] = ''
    right_now = datetime.strptime('2020.09.01', '%Y.%m.%d')
    for i, date_dict_str in enumerate(df['dates']):
        try:
            date_dict = ast.literal_eval(date_dict_str)
        except ValueError as e:
            date_dict_str = date_dict_str.replace('nan', '\'\'')
            print(f'{i}, {e}, in literal_eval')
        except SyntaxError as e:
            date_dict_str = date_dict_str + '\'}'
            print(f'{i}, {e}, in literal_eval')
            date_dict = ast.literal_eval(date_dict_str)
        
        timedelta_list = []
        for idx, date_str in date_dict.items():
            if '전' in date_str:
                if '시간' in date_str:
                    timedelta = 0
                elif '일' in date_str:
                    timedelta = int(date_str.split('일')[0])
                elif '달' in date_str:
                    timedelta = 30 * int(date_str.split('달')[0])
                elif '년' in date_str:
                    timedelta = 365 * int(date_str.split('년')[0])
                
            elif '일' in date_str:
                if not '년' in date_str:
                    date_str = '2020년 ' + date_str
                try:
                    timedelta = (right_now - datetime.strptime(date_str, '%Y년 %m월 %d일')).days
                except Exception as e:
                    print(f'{i}, {e}, [date_str] in "%Y년 %m월 %d일" strptime')
            else:
                try:
                    timedelta = (right_now - datetime.strptime(date_str, '%Y.%m.%d')).days
                except Exception as e:
                    print(f'{i}, {e}, [date_str] in "%Y.%m.%d" strptime')
                    if not date_str:
                        continue
                
            timedelta_list.append(timedelta)
        
        df.loc[i, f'{site}_1Y_num'] = len([x for x in timedelta_list if x <= 365])
        df.loc[i, f'{site}_6M_num'] = len([x for x in timedelta_list if x <= 180])
        df.loc[i, f'{site}_3M_num'] = len([x for x in timedelta_list if x <= 90])
        df.loc[i, f'{site}_1M_num'] = len([x for x in timedelta_list if x <= 30])
    return df

In [377]:
naver_review = preprocess_datetime(naver_review, 'naver')

24, EOL while scanning string literal (<unknown>, line 1), in literal_eval


In [378]:
kakao_review = preprocess_datetime(kakao_review, 'kakao')

4, time data '' does not match format '%Y.%m.%d', [date_str] in "%Y.%m.%d" strptime
4, time data '' does not match format '%Y.%m.%d', [date_str] in "%Y.%m.%d" strptime
4, time data '' does not match format '%Y.%m.%d', [date_str] in "%Y.%m.%d" strptime
4, time data '' does not match format '%Y.%m.%d', [date_str] in "%Y.%m.%d" strptime
4, time data '' does not match format '%Y.%m.%d', [date_str] in "%Y.%m.%d" strptime
6, time data '' does not match format '%Y.%m.%d', [date_str] in "%Y.%m.%d" strptime
6, time data '' does not match format '%Y.%m.%d', [date_str] in "%Y.%m.%d" strptime
6, time data '' does not match format '%Y.%m.%d', [date_str] in "%Y.%m.%d" strptime
6, time data '' does not match format '%Y.%m.%d', [date_str] in "%Y.%m.%d" strptime
7, time data '' does not match format '%Y.%m.%d', [date_str] in "%Y.%m.%d" strptime


In [379]:
dinning_review = preprocess_datetime(dinning_review, 'dinning')

23, malformed node or string: <_ast.Name object at 0x121610b00>, in literal_eval


In [380]:
google_review = preprocess_datetime(google_review, 'google')

In [381]:
mango_review = preprocess_datetime(mango_review, 'mango')

In [382]:
mango_review = mango_review.rename(columns = {'views':'mango_views', 'likes':'mango_likes', 'review_score':'mango_score'})

In [384]:
mango_review = mango_review.drop(['scores'], axis=1)

In [386]:
mango_review = mango_review.rename(columns = {'review_count':'mango_review_count', 'allreviews':'mango_allreviews', 'dates':'mango_dates', 'reviews':'mango_reviews', 'review_len':'mango_review_len' })

In [396]:
google_review = google_review.rename(columns={old:'google_'+old for old in google_review.columns[2:9]})

In [400]:
naver_review = naver_review.drop(['address', 'link', 'loc_name'], axis=1)

In [403]:
naver_review = naver_review.rename(columns={old:'naver_'+old for old in naver_review.columns[2:7]})

In [405]:
naver_review = naver_review.drop('naver_visitor_num', axis=1)

In [407]:
kakao_review = kakao_review.drop(['address', 'link', 'loc_name'], axis=1)

In [410]:
kakao_review = kakao_review.rename(columns={old:'kakao_'+old for old in kakao_review.columns[2:6]})

In [415]:
dinning_review = dinning_review.rename(columns={old:'dinning_'+old for old in dinning_review.columns[2:5]})

In [422]:
merge1 = pd.merge(naver_review, kakao_review, on=['name','rank'], how='outer')

In [424]:
merge2 = pd.merge(merge1, dinning_review, on=['name','rank'], how='outer')

In [427]:
merge2.head()

,rank,name,naver_reviews,naver_scores,naver_dates,naver_visit_num,num_review,naver_1Y_num,naver_6M_num,naver_3M_num,...,kakao_6M_num,kakao_3M_num,kakao_1M_num,dinning_review,dinning_dates,dinning_score,dinning_1Y_num,dinning_6M_num,dinning_3M_num,dinning_1M_num
0,1,스시아라타,"{0: '친절하시고 맛있어요! 다만, 저한텐 양이 좀 많았어요.', 1: '섬세함이...","{0: 5.0, 1: 4.5, 2: 5.0, 3: 5.0, 4: 5.0, 5: 5....","{0: '2020.08.06', 1: '2020.07.30', 2: '2020.07...","{0: '1번째 방문영수증', 1: '1번째 방문영수증', 2: '1번째 방문영수증...",9,8,8,3,...,6,3,2,{0: '#스시아라타\n#すし あらた\n#영등포스시아라타\n#아라타\n.\n벌써 시...,{0: '2019년 4월 11일'},{0: 5},0,0,0,0
1,2,오헨,"{0: '빵은 맛있는데 비싼게 흠이네요 버터프레첼 맛있어요', 1: '크로와상 일품...","{0: 4.5, 1: 4.0, 2: 5.0, 3: 5.0, 4: 5.0, 5: 5....","{0: '2020.08.24', 1: '2020.08.13', 2: '2020.07...","{0: '1번째 방문영수증', 1: '1번째 방문영수증', 2: '1번째 방문영수증...",14,14,12,7,...,1,1,0,{0: '위치가 인기있는 장소에서 살짝 비껴 있다. 하지만 빵의 수준은 아주 좋다....,"{0: '2018년 9월 8일', 1: '2018년 2월 3일', 2: '6월 9일...","{0: 5, 1: 4, 2: 5, 3: 5}",2,2,1,0
2,3,하쯔호,{0: '런치도 맛있네요! 첫점으로 나온 히라메 먹고 남친이랑 다담주 디너 예약 결...,"{0: 5.0, 1: 5.0, 2: 5.0, 3: 2.0, 4: 1.0, 5: 5....","{0: '2020.08.28', 1: '2020.08.27', 2: '2020.08...","{0: '2번째 방문영수증', 1: '1번째 방문영수증', 2: '1번째 방문영수증...",20,18,15,10,...,2,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,피그인더가든,"{0: '피그인더가든 마음 편한 곳^^', 1: '맛은 좋은데 서비스는 별로에요',...","{0: 4.0, 1: 2.5, 2: 4.5, 3: 3.5, 4: 5.0, 5: 4....","{0: '2020.08.31', 1: '2020.08.30', 2: '2020.08...","{0: '1번째 방문영수증', 1: '1번째 방문영수증', 2: '1번째 방문영수증...",204,181,147,102,...,10,6,4,{0: '여의도에 위치한 샐러드 맛집이에요.\n비건은 아니지만 건강한 먹거리를 추구...,"{0: '2019년 11월 26일', 1: '3월 1일', 2: '5월 1일', 3...","{0: 5, 1: 5, 2: 3, 3: 4, 4: 4, 5: 4}",5,2,1,1
4,5,정인면옥,{0: '면이 질기고 덜 익어서 먹으면 체할 정도. 카운터 젊은 사장님에게 얘기하니...,"{0: 0.5, 1: 5.0, 2: 2.0, 3: 4.5, 4: 4.0, 5: 5....","{0: '2020.08.30', 1: '2020.08.29', 2: '2020.08...","{0: '1번째 방문영수증', 1: '1번째 방문영수증', 2: '1번째 방문영수증...",160,140,130,68,...,20,16,5,{0: '여의도에서 평양냉면을 먹을때 무조건 원픽. 깔끔하고 맛있고 엄청 친절하다....,"{0: '7월 18일', 1: '5일 전', 2: '2019년 6월 29일', 3:...","{0: 5, 1: 5, 2: 3, 3: 5, 4: 4}",2,2,2,1
